# Profit Maximization with Price Elasticity of Demand

[![Investment_project.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity.ipynb)](https://colab.ampl.com)

This model finds the point that maximizes profit based on price elasticity and cost.


[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: Lessons, Managment, ampl, MIP, 

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

# 1. Download Necessary Extensions and Libraries

In [1]:
# Install dependencies
%pip install -q amplpy pandas
import pandas as pd                 # Loading panda to work with pandas.DataFrame objects (https://pandas.pydata.org/)
import numpy as np                  # Loading numpy to perform multidimensional calculations numpy.matrix (https://numpy.org/)

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

'# Google Colab & Kaggle integration\nfrom amplpy import AMPL, ampl_notebook\n\nampl = ampl_notebook(\n    modules=["cbc", "highs", "gurobi"],  # modules to install\n    license_uuid="default",  # license to use\n)  # instantiate AMPL object and register magics'

# 2. Model description

This model is designed to select one interpolated data point from a set of points in such a way that the total profit is maximized. The profit for each point is determined by the precomputed revenue (calculated as demand multiplied by price) minus the cost associated with fulfilling the demand for that point.

### Key Components:

- ***Demand and Price***: These values represent the demand and price at each data point, respectively.
- ***Revenue***: This is precomputed as the product of demand and price for each point.
- ***Cost***: A fixed cost associated with fulfilling the demand for any selected point.
- ***Decision Variables***: $select[p]$: A binary variable indicating whether point 𝑝 is selected (1) or not (0).

### Objective Function:
The total profit is maximized, defined as:

$Maximize TotalProfit = \sum_{𝑝 \in POINTS}(revenue[𝑝]−demand[𝑝]×cost)×select[𝑝]$

Where:

$revenue[p]=demand[p]×price[p]$

$cost=2 (fixed cost)$

### Constraints:

OnePointSelected: Exactly one point must be selected, ensuring that the total of all selected points equals 1:

$∑_{𝑝∈ POINTS} select[𝑝]=1$

### Interpolation Setup:
 
The data points for demand and price are interpolated linearly between given points using the ***scipy.interpolate.interp1d*** function. The $X$ values (demand) and $Y$ values (price) are used to generate new interpolated points, and the product 
$𝑋×𝑌$ is calculated for each new point to represent the revenue.

This model ensures that the most profitable point is selected based on the available demand, price, and cost data, providing an optimal point selection for maximum profit.

# 3. AMPL Model Formulation

In [4]:
%%writefile demand_elasticity_model.mod
reset;
### Sets and Parameters
set POINTS;                             # Set of interpolated points
param demand{POINTS};                   # Demand values for each point
param price{POINTS};                    # Price values for each point
param revenue{POINTS};                  # Revenue (Precomputed demand * price) values for each point
param cost = 2;                         # Fixed costs

### Decision Variables
var select{POINTS}, binary;             # Binary decision variable to select a point with demand & price

### Objective Function: Maximize Profit
maximize TotalProfit:
    sum {p in POINTS} (revenue[p] - demand[p] * cost) * select[p];

# Constraints
s.t. OnePointSelected:
    sum {p in POINTS} select[p] = 1;    # Exactly one point is selected

Overwriting demand_elasticity_model.mod


# 4. Load data

In [6]:
from scipy.interpolate import interp1d
ampl.read('demand_elasticity_model.mod')                         # Load the AMPL model from the file

# Define the data points as (X, Y) pairs
data_points = [(1, 10), (3, 7), (5, 5), (7, 4), (9, 3.8), (11, 4), (13, 3), (15, 1), (17, 0.5), (19, 0)]
x_points, y_points = zip(*data_points)                                  # Extract X and Y values from the data points
linear_interp = interp1d(x_points, y_points, kind='linear')             # Create the linear interpolator

# Generate new points and compute X*Y
x_new = np.arange(min(x_points), max(x_points) + 1)                     # Integer X values
y_new = linear_interp(x_new)                                            # Interpolated Y values
xy_new = x_new * y_new                                                  # Compute X * Y for each new point
interpolated_points = [(x, y, xy) for x, y, xy in zip(x_new, y_new, xy_new)]  # Combine the new points and X*Y values

# Set data for AMPL
ampl.set['POINTS'] = list(range(len(interpolated_points)))
ampl.param['demand'] = {i: p[0] for i, p in enumerate(interpolated_points)}
ampl.param['price'] = {i: p[1] for i, p in enumerate(interpolated_points)}
ampl.param['revenue'] = {i: p[2] for i, p in enumerate(interpolated_points)}


# Solve the model with solver options
ampl.option['show_stats'] = 1
ampl.option['display_1col'] = 0
ampl.option['omit_zero_rows'] = 1
ampl.option['omit_zero_cols'] = 1
ampl.option['mp_options'] = 'outlev=1 lim:time=20'
ampl.solve(solver='cplex', verbose=False)

# 5. Solve problem

In [7]:
# Set the solver type for use in solving the problems
solver = 'cplex'  # Use CBC solver for optimization tasks

ampl.option['show_stats'] = 0 # Show problem size statistics (default: 0)
ampl.option['display_1col'] = 0 # Disable single-column data display
#ampl.option['omit_zero_rows'] = 1 # Hide rows with zero values
#ampl.option['omit_zero_cols'] = 1 # Hide columns with zero values
ampl.option['mp_options'] = 'outlev=1 lim:time=20'   # Configure CBC options (output level and time limit)

ampl.solve(solver=solver, verbose=False)   # Solve the optimization problem using CBC solver  

## 6. Display results

In [16]:
# Display results for key variables
ampl.display('_varname', '_var', '_var.lb', '_var.ub', '_var.rc', '_var.slack')
ampl.display('_conname', '_con', '_con.body', '_con.lb', '_con.ub', '_con.slack')
ampl.display('_objname', '_obj')

:      _varname   _var _var.lb _var.ub _var.rc _var.slack    :=
1    'select[0]'    0      0       1       8        0
2    'select[1]'    0      0       1      13        0
3    'select[2]'    0      0       1      15        0
4    'select[3]'    0      0       1      16        0
5    'select[4]'    0      0       1      15        0
6    'select[5]'    0      0       1      15        0
7    'select[6]'    0      0       1      14        0
8    'select[7]'    0      0       1      15.2      0
9    'select[8]'    0      0       1      16.2      0
10   'select[9]'    0      0       1      19        0
11   'select[10]'   1      0       1      22        0
12   'select[11]'   0      0       1      18        0
13   'select[12]'   0      0       1      13        0
14   'select[13]'   0      0       1       0        0
15   'select[14]'   0      0       1     -15        0
16   'select[15]'   0      0       1     -20        0
17   'select[16]'   0      0       1     -25.5      0
18   'select[17]' 

# 7. Retrieve solution in Python

In [10]:
# Initialize an empty dictionary to store AMPL variable data
amplvar = dict()

# Prepare a list of AMPL variables
list_of_ampl_variables = [item[0] for item in ampl.get_variables()]

# Iterate over each variable name in the list
for key_ampl in list_of_ampl_variables:
    # Skip certain variables that are not to be processed (these variables won't be included in the output)
    if key_ampl not in ['']:
        # Convert the AMPL variable data to a pandas DataFrame
        df = ampl.var[key_ampl].to_pandas()
        # Filter the DataFrame to include only rows where the variable's value is greater than a small threshold (1e-5)
        filtered_df = df[df[f"{key_ampl}.val"] > 1e-5]
        # Round the values in the DataFrame to two decimal places
        rounded_df = filtered_df.round(2)
        # Convert the filtered DataFrame to a dictionary and add it to the amplvar dictionary
        amplvar[key_ampl] = rounded_df #.to_dict(orient='records')
print (amplvar[key_ampl])

    select.val
10           1
